In [1]:
# ===== 1. Imports =====
import ast
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# ===== 2. Load CSV and build multi-label targets =====
df = pd.read_csv("train_df.csv")
test_df = pd.read_csv("test_df.csv")

# Tags we focus on
TARGET_TAGS = ['math', 'graphs', 'strings', 'number theory',
               'trees', 'geometry', 'games', 'probabilities']


In [ ]:
!pip install -U wandb

In [ ]:
import wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jradi-ahmed (jradi-ahmed-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "eval_micro_f1",
        "goal": "maximize",
    },
    "parameters": {
        "learning_rate": {
            "values": [1e-5, 2e-5, 3e-5, 5e-5],
        },
        "train_batch_size": {
            "values": [8, 16],
        },
        "eval_batch_size": {
            "values": [16, 32],
        },
        "num_epochs": {
            "values": [5, 8, 12],
        },
        "weight_decay": {
            "values": [0.0, 0.01, 0.1],
        },
    },
}


In [ ]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'eval_micro_f1'},
 'parameters': {'eval_batch_size': {'values': [16, 32]},
                'learning_rate': {'values': [1e-05, 2e-05, 3e-05, 5e-05]},
                'num_epochs': {'values': [5, 8, 12]},
                'train_batch_size': {'values': [8, 16]},
                'weight_decay': {'values': [0.0, 0.01, 0.1]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="test_technique")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Create sweep with ID: lxcfaqiu
Sweep URL: https://wandb.ai/jradi-ahmed-none/test_technique/sweeps/lxcfaqiu


In [3]:

def parse_tags(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    elif isinstance(x, list):
        return x
    else:
        return []

# If you already have train_df and test_df separately, make sure BOTH have tag_list and tag columns:

for df_part in [df, test_df]:
    df_part["tag_list"] = df_part["tags_filtered"].apply(parse_tags)
    for tag in TARGET_TAGS:
        df_part[tag] = df_part["tag_list"].apply(lambda tags: 1 if tag in tags else 0)


In [4]:
# ===== 3. Build labels matrix and compute pos_weight =====

labels_np = df[TARGET_TAGS].values.astype(np.float32)  # shape (N, 8)
num_samples, num_labels = labels_np.shape
print("Labels shape:", labels_np.shape)

pos_counts = labels_np.sum(axis=0)                  # positives per label
neg_counts = num_samples - pos_counts               # negatives per label

# Avoid division by zero
pos_counts_safe = np.where(pos_counts == 0, 1, pos_counts)

pos_weight_np = neg_counts / pos_counts_safe        # shape (8,)
print("pos_counts:", pos_counts)
print("neg_counts:", neg_counts)
print("pos_weight_np:", pos_weight_np)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pos_weight = torch.tensor(pos_weight_np, dtype=torch.float32).to(device)

Labels shape: (2147, 8)
pos_counts: [1126.  434.  338.  280.  259.  133.   84.   74.]
neg_counts: [1021. 1713. 1809. 1867. 1888. 2014. 2063. 2073.]
pos_weight_np: [ 0.90674955  3.9470046   5.352071    6.667857    7.289575   15.142858
 24.559525   28.013514  ]


In [5]:
!pip install iterative-stratification

In [6]:
# ===== 4. Convert to Hugging Face Dataset and split =====
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
# Keep only cols we need + labels
keep_cols = ["description_clean", "code_clean"] + TARGET_TAGS
df_hf = df[keep_cols].copy()


# X = features (we just use the text columns as placeholders)
X = df_hf[["description_clean", "code_clean"]].values
# y = multi-label targets
y = df_hf[TARGET_TAGS].values.astype(int)

msss = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42,
)

for train_idx, test_idx in msss.split(X, y):
    df_train = df_hf.iloc[train_idx].reset_index(drop=True)
    df_test  = df_hf.iloc[test_idx].reset_index(drop=True)

print("Train label sums:", df_train[TARGET_TAGS].sum().values)
print("Test  label sums:", df_test[TARGET_TAGS].sum().values)

Train label sums: [900 347 270 224 207 106  67  59]
Test  label sums: [226  87  68  56  52  27  17  15]


In [7]:
train_ds = Dataset.from_pandas(df_train)
eval_ds  = Dataset.from_pandas(df_test)
train_full_ds = Dataset.from_pandas(df_hf)

# Add "labels" field = list of 8 ints/floats
def add_labels(example):
    example["labels"] = [example[tag] for tag in TARGET_TAGS]
    return example

train_ds = train_ds.map(add_labels)
eval_ds = eval_ds.map(add_labels)
train_full_ds = train_full_ds.map(add_labels)

Map:   0%|          | 0/1726 [00:00<?, ? examples/s]

Map:   0%|          | 0/421 [00:00<?, ? examples/s]

Map:   0%|          | 0/2147 [00:00<?, ? examples/s]

In [8]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
# ===== 5. Tokenizer & tokenization =====

model_name = "Mallard74/codebert-xcode-tags-classification"  # you can switch to another model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    texts = [
        (d or "") + "\n\n" + (c or "")
        for d, c in zip(batch["description_clean"], batch["code_clean"])
    ]
    return tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt",
    )

train_ds = train_ds.map(tokenize_fn, batched=True)
eval_ds  = eval_ds.map(tokenize_fn, batched=True)
train_full_ds = train_full_ds.map(tokenize_fn, batched=True)

# Set PyTorch format
train_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)
eval_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)
train_full_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

Map:   0%|          | 0/1726 [00:00<?, ? examples/s]

Map:   0%|          | 0/421 [00:00<?, ? examples/s]

Map:   0%|          | 0/2147 [00:00<?, ? examples/s]

In [10]:
# ===== 6. Model definition =====

config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification",
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config,
    )

In [11]:
# ===== 7. Custom Trainer with BCEWithLogitsLoss + pos_weight =====

class MultilabelTrainer(Trainer):
    def __init__(self, pos_weight=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss


In [12]:
def compute_metrics(eval_pred):
        logits, labels = eval_pred
        logits = torch.tensor(logits)
        labels = torch.tensor(labels)

        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).int().cpu().numpy()
        labels = labels.int().cpu().numpy()

        micro_f1 = f1_score(labels, preds, average="micro", zero_division=0)
        macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)

        return {"micro_f1": micro_f1, "macro_f1": macro_f1}

In [13]:
from transformers import EarlyStoppingCallback

In [ ]:
def train():
    with wandb.init():
        # Get sweep config from wandb
        sweep_config = wandb.config

        training_args = TrainingArguments(
            output_dir="/kaggle/working/finetune/",
            per_device_train_batch_size=sweep_config.train_batch_size,
            per_device_eval_batch_size=sweep_config.eval_batch_size,
            learning_rate=sweep_config.learning_rate,
            num_train_epochs=sweep_config.num_epochs,
            weight_decay=sweep_config.weight_decay,
            eval_strategy="epoch",
            save_strategy="epoch",
            report_to="wandb",
            logging_dir="/kaggle/working/tb_logs",
            load_best_model_at_end=True,
            logging_steps=10,
            run_name="test_study",
            metric_for_best_model="eval_micro_f1",
            greater_is_better=True,
        )

        trainer = MultilabelTrainer(
            model_init=model_init,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            tokenizer=tokenizer,
            pos_weight=pos_weight,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
        )

        # Train the model
        trainer.train()


In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: doe5escr with config:
wandb: 	eval_batch_size: 32
wandb: 	learning_rate: 2e-05
wandb: 	num_epochs: 8
wandb: 	train_batch_size: 16
wandb: 	weight_decay: 0.01
wandb: Currently logged in as: jradi-ahmed (jradi-ahmed-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/tmp/ipython-input-1662334917.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1
1,0.602400,0.732088,0.688172,0.595392
2,0.737700,0.628445,0.668567,0.623518
3,0.585800,0.599766,0.689757,0.649585
4,0.425600,0.619518,0.705058,0.663326
5,0.355700,0.559841,0.736146,0.684850
6,0.353200,0.603288,0.747725,0.700333
7,0.276100,0.617007,0.770026,0.724974
8,0.269700,0.613871,0.761905,0.712277


...

eval/loss,█▄▄▄▁▃▆▇▆▇▇▇
eval/macro_f1,▁▂▃▄▅▆▇█▇█▇▇
eval/micro_f1,▃▁▂▃▅▅▇█▇▇▇▇
eval/runtime,▁▄█▆▁▅▅▃▃▂▅█
eval/samples_per_second,█▅▁▃█▄▄▆▆▇▄▁
eval/steps_per_second,█▆▁▄█▅▅▆▆▇▄▂
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▂▃▄▄▄▃▄▂▂▂▅▂▁▆▁▂█▂▃▄▂▂▂▂▂▃▁▅█▃▁▄▃▂▂▂▂▂▂▂
train/learning_rate,█████▇▇▇▆▆▆▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
+1,...


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: h3vkdak4 with config:
wandb: 	eval_batch_size: 32
wandb: 	learning_rate: 3e-05
wandb: 	num_epochs: 5
wandb: 	train_batch_size: 16
wandb: 	weight_decay: 0


/tmp/ipython-input-1662334917.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1
1,0.609700,0.722014,0.692432,0.607240
2,0.694000,0.654612,0.669670,0.613925
3,0.556900,0.628516,0.692595,0.653849
4,0.452800,0.618954,0.722311,0.674795
5,0.331000,0.604364,0.744646,0.697644


eval/loss,█▄▂▂▁
eval/macro_f1,▁▂▅▆█
eval/micro_f1,▃▁▃▆█
eval/runtime,▁▆█▁▅
eval/samples_per_second,█▃▁█▄
eval/steps_per_second,█▂▁█▄
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▁▁▁▂▁▁▁▂▁▅▃▁▂▅▃▁▁▂▇▂█▃▁▇▄▄▁▂▃▄▃▂▂▃▄▅▂▁▂▂
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁
+1,...


In [ ]:
api = wandb.Api()
sweep = api.sweep(f"jradi-ahmed-none/test_technique/{"lxcfaqiu"}")

best_run = sweep.best_run()
print(f"Best Run ID: {best_run.id}")
print(f"Best Metrics: {best_run.summary.get('eval_micro_f1')}")
print("Best Hyperparameters:")
import pprint
pprint.pprint(best_run.config)

best_config = best_run.config

wandb: Sorting runs by -summary_metrics.eval_micro_f1


Best Run ID: doe5escr
Best Metrics: None
Best Hyperparameters:
{'_name_or_path': 'Mallard74/codebert-xcode-tags-classification',
 'accelerator_config': {'dispatch_batches': None,
                        'even_batches': True,
                        'gradient_accumulation_kwargs': None,
                        'non_blocking': False,
                        'split_batches': False,
                        'use_seedable_sampler': True},
 'adafactor': False,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'add_cross_attention': False,
 'architectures': ['RobertaForSequenceClassification'],
 'attention_probs_dropout_prob': 0.1,
 'auto_find_batch_size': False,
 'average_tokens_across_devices': True,
 'bad_words_ids': None,
 'batch_eval_metrics': False,
 'begin_suppress_tokens': None,
 'bf16': False,
 'bf16_full_eval': False,
 'bos_token_id': 0,
 'chunk_size_feed_forward': 0,
 'classifier_dropout': None,
 'cross_attention_hidden_size': None,
 'data_seed': None,
 'dataloader_

In [30]:
config = {
    "learning_rate": 2e-5,
    "train_batch_size": 16,
    "eval_batch_size": 32,
    "num_epochs": 8,
    "weight_decay": 0.01,
}

print(f"Training with manual config: {config}")

# Set up TrainingArguments with the manual config
training_args = TrainingArguments(
    output_dir="./hp_config_model_checkpoints",
    per_device_train_batch_size=config['train_batch_size'],
    per_device_eval_batch_size=config['eval_batch_size'],
    learning_rate=config['learning_rate'],
    num_train_epochs=config['num_epochs'],
    weight_decay=config['weight_decay'],
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_micro_f1",
    greater_is_better=True,
    report_to="none",
)

# Initialize a fresh model
manual_model = model_init()

# Initialize the custom trainer
trainer = MultilabelTrainer(
    model=manual_model,
    args=training_args,
    train_dataset=train_full_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    pos_weight=pos_weight,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

# Train
trainer.train()

# Save the final model
print("Saving model to ./hp_config_model ...")
trainer.save_model("./hp_config_model_best_run_v3")
tokenizer.save_pretrained("./hp_config_model_best_run_v3")
print("Done!")

Training with manual config: {'learning_rate': 2e-05, 'train_batch_size': 16, 'eval_batch_size': 32, 'num_epochs': 8, 'weight_decay': 0.01}


/tmp/ipython-input-1662334917.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Micro F1,Macro F1
1,No log,0.615743,0.667143,0.623609
2,No log,0.484121,0.733436,0.674414
3,No log,0.406615,0.770465,0.729751
4,0.632900,0.336786,0.800000,0.788913
5,0.632900,0.311820,0.832192,0.841742
6,0.632900,0.267547,0.844560,0.854670
7,0.632900,0.243928,0.862191,0.872296
8,0.332900,0.237402,0.863958,0.880757


Saving model to ./hp_config_model ...
Done!
